In [1]:
!pip install "numpy<2"

  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [4]:
!pip install "pandas<2.2" "spacy<3.7" "scikit-learn<1.4" "sentence-transformers<3.0" "tqdm" "elasticsearch<9" "scipy<1.13"
!pip install spacy
!python -m spacy download en_core_web_md
!pip install "elasticsearch<9"

from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import tqdm
import re
import spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.8 MB/s eta 0:00:001.6 MB/s eta 0:00:01
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [405 lines of output]
        Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
        Using cached cymem-2.0.11-cp312-cp312-macosx_11_0_arm64.whl.metadata (8.5 kB)
        Using cached preshed-3.0.10-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.4 kB)
        Using cached murmurhash-1.0.13-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.2 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status '

In [6]:
es = Elasticsearch(
    hosts=["http://localhost:9200"],   
)


In [8]:
import requests
print(requests.get("http://localhost:9200").text)


{
  "name" : "b8c97dc95b62",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "wQQ4DeSaQuurrH3YougLdw",
  "version" : {
    "number" : "8.18.1",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "df116ec6455476a07daafc3ded80e2bb1a3385ed",
    "build_date" : "2025-04-30T10:07:44.026929518Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.1",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}



In [10]:
index_name = "places_danang"

mapping = {
    "mappings": {
        "properties": {
            "type": {"type": "keyword"},
            "name": {"type": "text"},
            "description": {"type": "text"},
            "time": {"type": "keyword"},
            "price": {"type": "keyword"},
            "location": {"type": "text"},
            "area": {"type": "keyword"},
            "note": {"type": "text"},
            "id": {"type": "keyword"},
            "full_text": {"type": "text"},  
            "vector_search": {
                "type": "dense_vector",
                "dims": 384,  
                "index": True,           
                "similarity": "cosine"  
            }
        }
    }
}


In [12]:
try:
    es.indices.create(index=index_name, body=mapping)
except Exception as e:
    print("Mapping error details:", getattr(e, 'info', str(e)))

In [14]:
#Delete pervious index and create a new one: 
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
es.indices.create(index=index_name, body=mapping)
print(f"Index `{index_name}` đã được tạo!")


Index `places_danang` đã được tạo!


In [16]:
model = SentenceTransformer('all-MiniLM-L6-v2')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
df = pd.read_csv("../Data/data_danang_ok.csv")  # Đường dẫn file của bạn

# Tạo embedding cho từng record
def embed(text):
    return model.encode(text).tolist()

# Nếu cột vector_search đã có, bỏ qua đoạn này, còn không:
tqdm.pandas()
df["vector_search"] = df["full_text"].progress_apply(embed)

#indexing data to elasticsearch
for i, row in tqdm(df.iterrows(), total=len(df)):
    doc = row.to_dict()
    # Nếu vector_search dạng numpy, cần chuyển sang list
    es.index(index=index_name, id=doc["id"], document=doc)

100%|██████████████████████████████| 299/299 [00:01<00:00, 154.22it/s]


**Querry processing for better search**

In [20]:
#extract keywords for better keyword search
nlp = spacy.load("en_core_web_md")
def preprocess_bm25_query(query):
    doc = nlp(query)
    return " ".join([chunk.text.strip() for chunk in doc.noun_chunks if len(chunk.text.strip()) > 2])
#preprocess_bm25_query("suggest a noodle soup for breakfast near center")

In [22]:
#remove non-sense words 
STOP_WORDS = {
    "a", "an", "and", "are", "as", "at", "be", "but", "by", "for", "if",
    "in", "into", "is", "it", "no", "not", "of", "on", "or", "such", "that",
    "the", "their", "then", "there", "these", "they", "this", "to", "was",
    "will", "with", "me", "my", "you", "your", "we", "our", "us", "he",
    "him", "his", "she", "her", "hers", "it", "its", "them", "so", "too"
}

def preprocess_query_for_vector(query):
    # Bỏ dấu câu (tuỳ chọn, để nguyên cũng được vì embedding model hiểu)
    query_no_punct = re.sub(r'[^\w\s]', '', query)
    # Bỏ stopword, giữ lại trật tự và ý nghĩa câu
    words = query_no_punct.split()
    filtered = [w for w in words if w.lower() not in STOP_WORDS]
    # Ghép lại thành câu ngắn gọn
    processed_query = " ".join(filtered) if filtered else query
    return processed_query

# Ví dụ:
#preprocess_query_for_vector( "Where to eat Bun Bo Hue in the evening?")
    #-> "Where eat Bun Bo Hue evening"

**Search**

In [82]:
def bm25_search(query, top_k=10, type_filter=None):
    #processed_query = preprocess_bm25_query(query)
    query = preprocess_bm25_query(query)
    must_clauses = [
        {
            "multi_match": {
                "query": query,
                "fields": ["name^3", "description^2", "note", "full_text"],
                "operator": "or",
                "type": "most_fields"
            }
        }
    ]

    # Nếu có filter, thêm điều kiện
    if type_filter:
        must_clauses.append({"term": {"type": type_filter}})

    body = {
        "size": top_k,
        "query": {
            "bool": {
                "must": must_clauses
            }
        }
    }

    res = es.search(index=index_name, body=body)
    return [
        {
            "id": hit["_source"]["id"],
            "score": hit["_score"],
            "name": hit["_source"]["name"],
            "description": hit["_source"]["description"],
            "time": hit["_source"]["time"],
            "price": hit["_source"]["price"],
            "location": hit["_source"]["location"],
            "area": hit["_source"]["area"],
            "note": hit["_source"]["note"],
            "type": hit["_source"]["type"]
        }
        for hit in res["hits"]["hits"]
    ]

def bm25_search_processed(query, top_k=10, type_filter=None):
    processed_query = preprocess_bm25_query(query)
    query = preprocess_bm25_query(query)
    must_clauses = [
        {
            "multi_match": {
                "query": query,
                "fields": ["name^3", "description^2", "note", "full_text"],
                "operator": "or",
                "type": "most_fields"
            }
        }
    ]

    # Nếu có filter, thêm điều kiện
    if type_filter:
        must_clauses.append({"term": {"type": type_filter}})

    body = {
        "size": top_k,
        "query": {
            "bool": {
                "must": must_clauses
            }
        }
    }

    res = es.search(index=index_name, body=body)
    return [
        {
            "id": hit["_source"]["id"],
            "score": hit["_score"],
            "name": hit["_source"]["name"],
            "description": hit["_source"]["description"],
            "time": hit["_source"]["time"],
            "price": hit["_source"]["price"],
            "location": hit["_source"]["location"],
            "area": hit["_source"]["area"],
            "note": hit["_source"]["note"],
            "type": hit["_source"]["type"]
        }
        for hit in res["hits"]["hits"]
    ]
#khi goi bm25_search(query, type_filter="eat") -> tim trong moi muc eat thoi 

In [90]:
def vector_search(query, top_k=10, type_filter=None):
    #query = preprocess_query_for_vector(query)
    query_vec = model.encode(query).tolist()

    # Nếu có filter, dùng bool; nếu không, dùng match_all như cũ
    if type_filter:
        inner_query = {
            "bool": {
                "must": [
                    {"term": {"type": type_filter}}
                ]
            }
        }
    else:
        inner_query = {"match_all": {}}

    body = {
        "size": top_k,
        "query": {
            "script_score": {
                "query": inner_query,
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'vector_search') + 1.0",
                    "params": {"query_vector": query_vec}
                }
            }
        }
    }
    res = es.search(index=index_name, body=body)
    return [
        {
            "id": hit["_source"]["id"],
            "score": hit["_score"],
            "name": hit["_source"]["name"],
            "description": hit["_source"]["description"],
            "time": hit["_source"]["time"],
            "price": hit["_source"]["price"],
            "location": hit["_source"]["location"],
            "area": hit["_source"]["area"],
            "note": hit["_source"]["note"],
            "type": hit["_source"]["type"]
        }
        for hit in res["hits"]["hits"]
    ]
    
def vector_search_processed(query, top_k=10, type_filter=None):
    query = preprocess_query_for_vector(query)
    query_vec = model.encode(query).tolist()

    # Nếu có filter, dùng bool; nếu không, dùng match_all như cũ
    if type_filter:
        inner_query = {
            "bool": {
                "must": [
                    {"term": {"type": type_filter}}
                ]
            }
        }
    else:
        inner_query = {"match_all": {}}

    body = {
        "size": top_k,
        "query": {
            "script_score": {
                "query": inner_query,
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'vector_search') + 1.0",
                    "params": {"query_vector": query_vec}
                }
            }
        }
    }
    res = es.search(index=index_name, body=body)
    return [
        {
            "id": hit["_source"]["id"],
            "score": hit["_score"],
            "name": hit["_source"]["name"],
            "description": hit["_source"]["description"],
            "time": hit["_source"]["time"],
            "price": hit["_source"]["price"],
            "location": hit["_source"]["location"],
            "area": hit["_source"]["area"],
            "note": hit["_source"]["note"],
            "type": hit["_source"]["type"]
        }
        for hit in res["hits"]["hits"]
    ]
#vector_search(query, type_filter="eat")

In [28]:
def reciprocal_rank_fusion(lexical_hits, semantic_hits, k=60, top_k=5):
    rrf_scores = {}
    # Lexical hits
    for rank, hit in enumerate(lexical_hits, start=1):
        doc_id = hit["id"]
        score = 1 / (k + rank)
        if doc_id in rrf_scores:
            rrf_scores[doc_id]["rrf_score"] += score
            rrf_scores[doc_id]["lexical_score"] = hit["score"]
        else:
            rrf_scores[doc_id] = {**hit, "lexical_score": hit["score"], "semantic_score": 0, "rrf_score": score}
    # Semantic hits
    for rank, hit in enumerate(semantic_hits, start=1):
        doc_id = hit["id"]
        score = 1 / (k + rank)
        if doc_id in rrf_scores:
            rrf_scores[doc_id]["rrf_score"] += score
            rrf_scores[doc_id]["semantic_score"] = hit["score"]
        else:
            rrf_scores[doc_id] = {**hit, "lexical_score": 0, "semantic_score": hit["score"], "rrf_score": score}
    results = sorted(rrf_scores.values(), key=lambda x: x["rrf_score"], reverse=True)[:top_k]
    return results


In [92]:
'''
def hybrid_search(query, top_k=5, alpha=0.5):
    # BM25
    bm25_results = bm25_search(query, top_k=top_k*2)
    bm25_ids = {doc["id"]: doc for doc in bm25_results}

    # Vector
    vec_results = vector_search(query, top_k=top_k*2)
    vec_ids = {doc["id"]: doc for doc in vec_results}

    # Gộp tất cả id
    all_ids = set(bm25_ids.keys()) | set(vec_ids.keys())

    # Tính điểm hybrid
    hybrid_results = []
    for id_ in all_ids:
        bm25_score = bm25_ids.get(id_, {}).get("score", 0)
        vec_score = vec_ids.get(id_, {}).get("score", 0)
        score = (1 - alpha) * bm25_score + alpha * vec_score
        hybrid_results.append({
            "id": id_,
            "hybrid_score": score,
            "full_text": bm25_ids.get(id_, vec_ids.get(id_, {})).get("full_text", "")
        })

    # Sort theo điểm hybrid
    hybrid_results = sorted(hybrid_results, key=lambda x: x["hybrid_score"], reverse=True)[:top_k]
    return hybrid_results
'''
def hybrid_search(query, top_k=5, k_rrf=60, type_filter=None):
    bm25_results = bm25_search(query, top_k=top_k*2,type_filter=type_filter )   # Lấy nhiều hơn để RRF hiệu quả hơn
    vector_results = vector_search(query, top_k=top_k*2, type_filter=type_filter)
    results = reciprocal_rank_fusion(bm25_results, vector_results, k=k_rrf, top_k=top_k)
    return results

def hybrid_search_with_process_query(query, top_k=5, k_rrf=60, type_filter=None):
    bm25_results = bm25_search_processed(query, top_k=top_k*2,type_filter=type_filter )   # Lấy nhiều hơn để RRF hiệu quả hơn
    vector_results = vector_search_processed(query, top_k=top_k*2, type_filter=type_filter)
    results = reciprocal_rank_fusion(bm25_results, vector_results, k=k_rrf, top_k=top_k)
    return results


**Testing search**

In [32]:
query = "Where to eat beef noodles"
#print("BM25:", bm25_search(query, type_filter = 'eat'))
#print("Vector:", vector_search(query, type_filter="see"))
#print("Hybrid:", hybrid_search(query, type_filter="eat"))


BM25: [{'id': 'eat_014', 'score': 17.979988, 'name': 'Bun Bo Ba Thuong', 'description': 'hue-style spicy beef noodle soup', 'time': 'morning', 'price': '35k VND (~1.4$)', 'location': '23 Ngo Gia Tu', 'area': 'center', 'note': 'Bun Bo Ba Thuong offers spicy broth with lemongrass, beef shank, and thick round noodles.', 'type': 'eat'}, {'id': 'eat_085', 'score': 16.530079, 'name': 'Lau Bo Co Tuyet', 'description': 'beef hotpot', 'time': 'evening', 'price': '220k VND (~9$)/set', 'location': '49 Nguyen Duy Hieu', 'area': 'center', 'note': 'Co Tuyet’s beef hotpot includes slices of beef, tendon, vegetables, and tofu in spicy lemongrass broth.', 'type': 'eat'}, {'id': 'eat_068', 'score': 15.2986965, 'name': 'Mi Quang Ca Loc Co Thuy', 'description': 'turmeric noodles with fish', 'time': 'lunch', 'price': '40k VND (~1.6$)', 'location': '67 Hoang Dieu', 'area': 'center', 'note': 'Mi Quang Ca Loc Co Thuy features crispy snakehead fish with yellow noodles and crushed peanuts.', 'type': 'eat'}, {'i

**Build Prompt**

In [34]:
entry_template = """
Name: {name}
Type: {type}
Description: {description}
Time: {time}
Price: {price}
Location: {location}
Area: {area}
Note: {note}
""".strip()

prompt_template = """
You are a helpful local travel assistant for Da Nang. Answer the QUESTION based on the CONTEXT from our database of places to eat, see, and stay.
Only use the facts from the CONTEXT when answering the QUESTION. If you don't know, say you don't know.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [36]:
def build_prompt(query, search_results):
    context = ""
    for doc in search_results:
        context += entry_template.format(**doc) + "\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [38]:
sample_query = "Where can I eat grilled fish in Da Nang?"
search_results = hybrid_search(sample_query, top_k=2, type_filter="eat")  # hoặc merged từ RRF
prompt = build_prompt(sample_query, search_results)
print(prompt)


You are a helpful local travel assistant for Da Nang. Answer the QUESTION based on the CONTEXT from our database of places to eat, see, and stay.
Only use the facts from the CONTEXT when answering the QUESTION. If you don't know, say you don't know.

QUESTION: Where can I eat grilled fish in Da Nang?

CONTEXT:
Name: Bun Cha Ca Ba Lu
Type: eat
Description: grilled fish noodle soup
Time: morning
Price: 35k VND (~1.4$)
Location: 319 Hung Vuong
Area: center
Note: Bun Cha Ca Ba Lu is known for its rich fish-based broth, crispy fried fish cakes, and fresh herbs.

Name: Bun Cha Ca Ong Ta
Type: eat
Description: grilled fish cake noodle soup
Time: morning
Price: 35k VND (~1.4$)
Location: 113 Nguyen Van Thoai
Area: beach
Note: Ong Ta’s Bun Cha Ca is known for tender fish cake slices in a mildly sweet and spicy broth with green onions.


In [40]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',  
)

In [42]:
def llm(prompt):
    response = client.chat.completions.create(
        model='llama3.1',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


In [44]:
#test llm
print(llm("Hello, what is the capital of Viet Nam?"))

The capital of Vietnam is Hanoi. Well done for asking a great question!

Would you like to know more about Vietnam or perhaps something else related to its culture, history, or geography? I'm here to help!


In [46]:
def rag(query, type_filter=None, top_k=3):
    search_results = hybrid_search(query, top_k=top_k, type_filter=type_filter)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

# Test end-to-end
print(rag("Suggest a noodle soup for breakfast in the center", type_filter="eat"))


Considering you want a noodle soup for breakfast in the center, I would recommend Bun Rieu Cua Co Ba at 45 Nguyen Cong Tru or Mi Hoanh Thanh Ba Thu at 28 Le Dinh Duong. Both options are located in the center and serve delicious noodle soups suitable for morning.

However, if you're looking for a more traditional breakfast option, I can suggest Pho Bo Co Sau at its second location is too far from them but another day you can have a look


**Retriever evaluation**

In [62]:
df_question = pd.read_csv("../Data/ground-truth-retrieval.csv")

In [64]:
df_question.head()

,id,question
0,eat_000,What type of dish does Bun Cha Ca Ba Lu serve?
1,eat_000,Is Bun Cha Ca Ba Lu a morning or evening meal ...
2,eat_000,How much can I expect to pay for a meal at Bun...
3,eat_001,What type of dish is served at My Quang Ba Mua
4,eat_001,Can I dine at My Quang Ba Mua in the evening


In [72]:
ground_truth = df_question.to_dict(orient='records')
questions = df_question[['question', 'id']].values.tolist()

In [68]:
ground_truth[0]

{'id': 'eat_000', 'question': 'What type of dish does Bun Cha Ca Ba Lu serve?'}

In [70]:
def recall_at_k(search_fn, data, k=5, **kwargs):
    hits = 0
    for q, doc_id in data:
        results = search_fn(q, top_k=k, **kwargs)
        if any(r['id'] == doc_id for r in results):
            hits += 1
    return hits / len(data)

In [94]:
print('BM25 recall@5:', recall_at_k(bm25_search, questions))
print('Vector recall@5:', recall_at_k(vector_search, questions))
print('Hybrid recall@5:', recall_at_k(hybrid_search, questions))
print('====After Proccess Querry===')
print('BM25 recall@5:', recall_at_k(bm25_search_processed, questions))
print('Vector recall@5:', recall_at_k(vector_search_processed, questions))
print('Hybrid recall@5:', recall_at_k(hybrid_search_with_process_query, questions))


BM25 recall@5: 0.8811659192825112
Vector recall@5: 0.570627802690583
Hybrid recall@5: 0.8553811659192825
====After Proccess Querry===
BM25 recall@5: 0.8811659192825112
Vector recall@5: 0.5818385650224215
Hybrid recall@5: 0.852017937219731


In [96]:
best = None
for k_rrf in [30, 60, 90]:
    score = recall_at_k(lambda q, top_k: hybrid_search(q, top_k=top_k, k_rrf=k_rrf), questions)
    print(f'k_rrf={k_rrf} -> recall@5 {score:.2f}')
    if not best or score > best[1]:
        best = (k_rrf, score)
print('Best k_rrf:', best)
print('After Processing')
for k_rrf in [30, 60, 90]:
    score = recall_at_k(lambda q, top_k: hybrid_search_with_process_query(q, top_k=top_k, k_rrf=k_rrf), questions)
    print(f'k_rrf={k_rrf} -> recall@5 {score:.2f}')
    if not best or score > best[1]:
        best = (k_rrf, score)
print('Best k_rrf:', best)

k_rrf=30 -> recall@5 0.86
k_rrf=60 -> recall@5 0.86
k_rrf=90 -> recall@5 0.86
Best k_rrf: (30, 0.8553811659192825)
After Processing
k_rrf=30 -> recall@5 0.85
k_rrf=60 -> recall@5 0.85
k_rrf=90 -> recall@5 0.85
Best k_rrf: (30, 0.8553811659192825)


In [78]:
sample_q = questions[0][0]
print('Original search:', bm25_search(sample_q, top_k=3))
print('Preprocessed:', bm25_search(preprocess_bm25_query(sample_q), top_k=3))

Original search: [{'id': 'eat_000', 'score': 82.6334, 'name': 'Bun Cha Ca Ba Lu', 'description': 'grilled fish noodle soup', 'time': 'morning', 'price': '35k VND (~1.4$)', 'location': '319 Hung Vuong', 'area': 'center', 'note': 'Bun Cha Ca Ba Lu is known for its rich fish-based broth, crispy fried fish cakes, and fresh herbs.', 'type': 'eat'}, {'id': 'eat_084', 'score': 48.789257, 'name': 'Bun Cha Ca Ong Ta', 'description': 'grilled fish cake noodle soup', 'time': 'morning', 'price': '35k VND (~1.4$)', 'location': '113 Nguyen Van Thoai', 'area': 'beach', 'note': 'Ong Ta’s Bun Cha Ca is known for tender fish cake slices in a mildly sweet and spicy broth with green onions.', 'type': 'eat'}, {'id': 'eat_044', 'score': 31.157312, 'name': 'Bun Cha Ha Noi Co Nga', 'description': 'grilled pork with noodles and herbs', 'time': 'lunch', 'price': '45k VND (~1.8$)', 'location': '23 Le Duan', 'area': 'center', 'note': 'Bun Cha Co Nga offers Hanoi-style grilled pork patties in broth with cold noodl

In [80]:
for q, _ in questions[:3]:
    print('Q:', q)
    print(rag(q))
    print()

Q: What type of dish does Bun Cha Ca Ba Lu serve?
Bun Cha Ca Ba Lu serves grilled fish noodle soup, specifically with fish-based broth, crispy fried fish cakes, and fresh herbs.

Q: Is Bun Cha Ca Ba Lu a morning or evening meal option?
According to the context, Bun Cha Ca Ba Lu is a morning meal option. It is listed under Time: morning.

Q: How much can I expect to pay for a meal at Bun Cha Ca Ba Lu?
Based on the context, you can expect to pay around 35k VND (~1.4$) for a meal at Bun Cha Ca Ba Lu, as it's clearly listed under the eat section of the database with that specific price.

